In [1]:
import math
import os
import re
import altair as alt
import numpy as np
import pandas as pd
import scipy.stats
import yaml
import neutcurve
import scipy.stats
print(f"Using `neutcurve` version {neutcurve.__version__}")
import sys
import matplotlib.pyplot as plt

# allow more rows for Altair
_ = alt.data_transformers.disable_max_rows()

#import altair themes from /data/custom_analyses_data/theme.py and enable
import theme
alt.themes.register('main_theme', theme.main_theme)
alt.themes.enable('main_theme')
from pathlib import Path

repo_root = Path.cwd().parent
os.chdir(repo_root)
print(os.getcwd())
#print(os.listdir("raw_plate_reader"))plt.rcParams['svg.fonttype'] = 'none' #from bernadeta, for correct font svg output

Using `neutcurve` version 2.1.0
/fh/fast/bloom_j/computational_notebooks/tmcmahon/2024/02_RSV/RSV_evolution_neut


In [2]:
df = pd.read_csv('01_data/other/25.05.0405_RSVTiter.csv')
df['virus'] = df['virus'].str.strip()
print(df['virus'].unique())
df

print("Unique target cells:")
print(df['target cell'].unique())

print("\nUnique protocols:")
print(df['protocol'].unique())

print("\nUnique viruses:")
print(df['virus'].unique())

['Long old' 'Long F -4+G (full)' 'Bald' 'VSVG' 'ABJ'
 'Long F (full)+G (full)']
Unique target cells:
['293T-TIM1' 'A549' 'Hep2' 'Huh7.5.1']

Unique protocols:
['ours' 'Haid et al' 'Cui et al' 'Hu et al']

Unique viruses:
['Long old' 'Long F -4+G (full)' 'Bald' 'VSVG' 'ABJ'
 'Long F (full)+G (full)']


In [3]:
df.head

<bound method NDFrame.head of        virus protocol target cell  replicate   dilution          RLU       uL  \
0   Long old     ours   293T-TIM1           1         2  261000000.0  50.0000   
1   Long old     ours   293T-TIM1           1         4  154000000.0  25.0000   
2   Long old     ours   293T-TIM1           1         8   80800000.0  12.5000   
3   Long old     ours   293T-TIM1           1        16   43000000.0   6.2500   
4   Long old     ours   293T-TIM1           1        32   23000000.0   3.1250   
..       ...      ...         ...         ...       ...          ...      ...   
90  Long old     ours    Huh7.5.1           1         4     582000.0  25.0000   
91  Long old     ours    Huh7.5.1           1         8     284000.0  12.5000   
92  Long old     ours    Huh7.5.1           1        16     109000.0   6.2500   
93  Long old     ours    Huh7.5.1           1        32      84900.0   3.1250   
94  Long old     ours    Huh7.5.1           1        64      30300.0   1.5625  

In [4]:
# Define a function to configure the chart style
def configure_chart(chart):
    return chart.configure_axis(
        labelFontSize=14,
        titleFontSize=14,
        grid=False
    ).configure_view(
        strokeWidth=2
    )

# Define a selection for hover
hover = alt.selection_point(fields=["virus", "replicate"], nearest=True, on="mouseover", empty="none")

# Create the base chart
base = alt.Chart(df).encode(
    x=alt.X(
        'virus:N',
        title='Virus',
        axis=alt.Axis(labelAngle=45)
    ),
    y=alt.Y(
        'RLU/uL:Q',
        title='RLU/uL',
        scale=alt.Scale(type='log')  # Optional: Use log scale if needed
    ),   
    color=alt.Color(
        'replicate:N',
        title='Replicate',
        legend=alt.Legend(title="Replicate")
    ),
    tooltip=[
        alt.Tooltip('virus:N', title='Virus'),
        alt.Tooltip('target cell:N', title='Target Cell'),
        alt.Tooltip('replicate:N', title='Replicate'),
        alt.Tooltip('dilution:Q', title='Dilution'),
        alt.Tooltip('RLU:Q', title='RLU'),
        alt.Tooltip('uL:Q', title='uL'),
        alt.Tooltip('RLU/uL:Q', title='RLU/uL')
    ]
)

# Points and hover logic
points = base.mark_point(size=80, filled=True).add_params(hover)

# Add a rule (highlight nearest point)
highlight = base.transform_filter(hover).mark_circle(size=200, color="red")

# Combine points and highlight, and facet by 'target cell'
faceted_chart = (
    (points + highlight)
    .properties(width=300, height=300)
    .facet(
        facet=alt.Facet('target cell:N', title='Target Cell')
    )
)

# Configure and display the chart
final_chart = configure_chart(faceted_chart)
final_chart


alt.FacetChart(...)

## add error to new df 

In [5]:
# Compute mean and error bars
df_agg = df.groupby(['virus', 'target cell','protocol']).agg(
    mean_RLU_uL=('RLU/uL', 'mean'),
    std_RLU_uL=('RLU/uL', 'std')  # Standard deviation for error bars
).reset_index()

# Calculate upper and lower limits
df_agg['lower_RLU_uL'] = df_agg['mean_RLU_uL'] - df_agg['std_RLU_uL']
df_agg['upper_RLU_uL'] = df_agg['mean_RLU_uL'] + df_agg['std_RLU_uL']

# Display the final aggregated DataFrame
df_agg.head()

,virus,target cell,protocol,mean_RLU_uL,std_RLU_uL,lower_RLU_uL,upper_RLU_uL
0,ABJ,Hep2,Cui et al,51.250000,8.225023,43.024977,59.475023
1,Bald,A549,Haid et al,214.500000,50.204581,164.295419,264.704581
2,Bald,Hep2,Cui et al,51.616667,4.118940,47.497727,55.735607
3,Bald,Hep2,Haid et al,26.066667,3.113412,22.953255,29.180079
4,Bald,Huh7.5.1,Hu et al,199.666667,56.273143,143.393523,255.939810


In [6]:
virus_order = [
    'Bald', 'VSVG', 'ABJ', 'Long F -4+G (full)','Long F (full)+G (full)', 'Long old'
]

# Clean and verify virus names
df_agg['virus'] = df_agg['virus'].astype(str).str.strip()

# Map to numerical order
virus_order_dict = {virus: i for i, virus in enumerate(virus_order)}
df_agg['virus_order'] = df_agg['virus'].map(virus_order_dict)

# Check for missing mappings
print("Unmapped viruses:", df_agg[df_agg['virus_order'].isna()]['virus'].unique())

# Optional: convert to int only if safe
if df_agg['virus_order'].isna().sum() == 0:
    df_agg['virus_order'] = df_agg['virus_order'].astype(int)
    
df_agg.to_csv("03_output/processed_data/Past_RSV_psv_avg_RLU-uL.csv", index=False)


Unmapped viruses: []


## final plot fig 1

In [7]:
virus_order = [
    'Bald', 'VSVG', 'ABJ', 'Long F -4+G (full)','Long F (full)+G (full)', 'Long old'
]

print("Expected Order:", virus_order)
print("Unique Values in DataFrame:", df_agg['virus'].unique())

# Ensure consistent virus naming
df_agg['virus'] = df_agg['virus'].str.strip()

# Filter to only viruses in the list
df_agg = df_agg[df_agg['virus'].isin(virus_order)]

# Convert to categorical with the correct order
df_agg['virus'] = pd.Categorical(df_agg['virus'], categories=virus_order, ordered=True)

# Sort
df_agg = df_agg.sort_values('virus')

# Sanity check
print("Remaining unmapped viruses (should be empty):", set(df_agg['virus']) - set(virus_order))


Expected Order: ['Bald', 'VSVG', 'ABJ', 'Long F -4+G (full)', 'Long F (full)+G (full)', 'Long old']
Unique Values in DataFrame: ['ABJ' 'Bald' 'Long F (full)+G (full)' 'Long F -4+G (full)' 'Long old'
 'VSVG']
Remaining unmapped viruses (should be empty): set()


In [8]:
import pandas as pd
import altair as alt

# Define custom virus order
virus_order = [
    'Bald', 'VSVG','ABJ','Long F -4+G (full)','Long old', 'VSVG'
]

# Ensure category consistency and remove spaces
df_agg['virus'] = df_agg['virus'].str.strip()

# Create an explicit numerical order for sorting
virus_order_dict = {virus: i for i, virus in enumerate(virus_order)}
df_agg['virus_order'] = df_agg['virus'].map(virus_order_dict)

# Sort DataFrame before passing it to Altair
df_agg = df_agg.sort_values('virus_order')

# Define color and shape mappings
color_mapping = {
    "293T-TIM1": "#999999",   # Gray
    "A549": "#377eb8",        # Blue
    "Hep2": "#4daf4a",        # Green
    "Huh7.5.1": "#e41a1c"     # Red
}

shape_mapping = {
    "293T-TIM1": "circle",
    "A549": "square",
    "Hep2": "diamond",
    "Huh7.5.1": "triangle"
}

log_ticks = [10**i for i in range(1, 8)]  # 10^1 to 10^7
log_labels = ["10¹", "10²", "10³", "10⁴", "10⁵", "10⁶", "10⁷"]

y_label_expr = "{ " + ", ".join(f"{v}: '{label}'" for v, label in zip(log_ticks, log_labels)) + " }[datum.value]"

# Base chart with explicit sorting
base = alt.Chart(df_agg).encode(
    x=alt.X(
        'virus:O',  # ✅ Correct usage
        title='',
        axis=alt.Axis(
            labelAngle=270,
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold'
        ),
        sort=virus_order  # optional if you want explicit ordering
    ),
     y=alt.Y(
        'mean_RLU_uL:Q',
        title='Pseudovirus Titer (RLU/uL)',
        scale=alt.Scale(type='log', domain=[10, df_agg['upper_RLU_uL'].max()]),
        axis=alt.Axis(
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold',
            values=log_ticks,  # Force log ticks
            labelExpr="{ " + ", ".join(f"{v}: '{label}'" for v, label in zip(log_ticks, log_labels)) + " }[datum.value]"
        ),
    ),
    color=alt.Color(
        'target cell:N',
        scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    shape=alt.Shape(
        'target cell:N',
        scale=alt.Scale(domain=list(shape_mapping.keys()), range=list(shape_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    tooltip=[
        alt.Tooltip('virus:N', title='Virus'),
        alt.Tooltip('target cell:N', title='Target Cell'),
        alt.Tooltip('mean_RLU_uL:Q', title='Pseudovirus Titer (RLU/uL)'),
        alt.Tooltip('std_RLU_uL:Q', title='Standard Deviation')
    ]
)

# Error bars with black color (added first to appear behind)
error_bars = alt.Chart(df_agg).mark_rule(size=2, color='black').encode(
    x='virus:O',
    y='lower_RLU_uL:Q',
    y2='upper_RLU_uL:Q'
)

# Error bar caps with black color (added before points to stay behind)
error_caps = (
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus:O',
        y='lower_RLU_uL:Q'
    ) +
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus:O',
        y='upper_RLU_uL:Q'
    )
)

# Points for the mean with fixed shape encoding and black outline (added last to stay on top)
points = base.mark_point(size=80, filled=True, opacity=1, stroke='black', strokeWidth=1.5).add_params(hover)

# Highlight nearest point (still on top)
highlight = base.transform_filter(hover).mark_circle(size=200, color="red")

# Combine layers ensuring error bars & caps are behind points
combined_chart = (error_bars + error_caps + points + highlight).properties(
    width=300,
    height=200
).configure_axis(
    labelFontSize=8,  # Apply globally to be safe
    labelFontWeight='bold',
    titleFontSize=12,
    titleFontWeight='bold',
    grid=False
).configure_view(
    strokeWidth=2
)

combined_chart.save("03_output/plots/Titer_old-psv.html")

# Display the chart
combined_chart

alt.LayerChart(...)

## facet by protocol

In [9]:
import pandas as pd
import altair as alt

# Ensure category consistency and remove spaces
df_agg['virus'] = df_agg['virus'].str.strip()

# Create an explicit numerical order for sorting
virus_order_dict = {virus: i for i, virus in enumerate(virus_order)}
df_agg['virus_order'] = df_agg['virus'].map(virus_order_dict)

# Sort DataFrame before passing it to Altair
df_agg = df_agg.sort_values('virus_order')

# Define color and shape mappings
color_mapping = {
    "293T-TIM1": "#999999",   # Gray
    "A549": "#377eb8",        # Blue
    "Hep2": "#4daf4a",        # Green
    "Huh7.5.1": "#e41a1c"     # Red
}

shape_mapping = {
    "293T-TIM1": "circle",
    "A549": "square",
    "Hep2": "diamond",
    "Huh7.5.1": "triangle"
}

log_ticks = [10**i for i in range(1, 8)]  # 10^1 to 10^7
log_labels = ["10¹", "10²", "10³", "10⁴", "10⁵", "10⁶", "10⁷"]

y_label_expr = "{ " + ", ".join(f"{v}: '{label}'" for v, label in zip(log_ticks, log_labels)) + " }[datum.value]"

# Base chart with explicit sorting
base = alt.Chart(df_agg).encode(
    x=alt.X(
        'virus:O',  # ✅ Correct usage
        title='',
        axis=alt.Axis(
            labelAngle=270,
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold'
        ),
        sort=virus_order  # optional if you want explicit ordering
    ),
     y=alt.Y(
        'mean_RLU_uL:Q',
        title='Pseudovirus Titer (RLU/uL)',
        scale=alt.Scale(type='log', domain=[10, df_agg['upper_RLU_uL'].max()]),
        axis=alt.Axis(
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold',
            values=log_ticks,  # Force log ticks
            labelExpr="{ " + ", ".join(f"{v}: '{label}'" for v, label in zip(log_ticks, log_labels)) + " }[datum.value]"
        ),
    ),
    color=alt.Color(
        'target cell:N',
        scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    shape=alt.Shape(
        'target cell:N',
        scale=alt.Scale(domain=list(shape_mapping.keys()), range=list(shape_mapping.values())),
        legend=alt.Legend(title='Target Cell')  # KEEP LEGEND
    ),
    tooltip=[
        alt.Tooltip('virus:N', title='Virus'),
        alt.Tooltip('target cell:N', title='Target Cell'),
        alt.Tooltip('mean_RLU_uL:Q', title='Pseudovirus Titer (RLU/uL)'),
        alt.Tooltip('std_RLU_uL:Q', title='Standard Deviation')
    ]
)

# Error bars with black color (added first to appear behind)
error_bars = alt.Chart(df_agg).mark_rule(size=2, color='black').encode(
    x='virus:O',
    y='lower_RLU_uL:Q',
    y2='upper_RLU_uL:Q'
)

# Error bar caps with black color (added before points to stay behind)
error_caps = (
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus:O',
        y='lower_RLU_uL:Q'
    ) +
    alt.Chart(df_agg).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='virus:O',
        y='upper_RLU_uL:Q'
    )
)

# Points for the mean with fixed shape encoding and black outline (added last to stay on top)
points = base.mark_point(size=80, filled=True, opacity=1, stroke='black', strokeWidth=1.5).add_params(hover)

# Highlight nearest point (still on top)
highlight = base.transform_filter(hover).mark_circle(size=200, color="red")

# Combine layers ensuring error bars & caps are behind points
base_chart = (error_bars + error_caps + points + highlight).properties(
    width=300,
    height=200
)

# Apply faceting
faceted_chart = base_chart.facet(
    column=alt.Column(
        'protocol:N',
        title=None,
        header=alt.Header(labelFontSize=14, labelFontWeight='bold')
    )
).configure_axis(
    labelFontSize=8,
    labelFontWeight='bold',
    titleFontSize=12,
    titleFontWeight='bold',
    grid=False
).configure_view(
    strokeWidth=2
)

# Save and display
faceted_chart.save("03_output/plots/Titer_old-psv_faceted-by-protocol.html")
faceted_chart

alt.FacetChart(...)

## plot seperately 

In [10]:
# Read in the saved file
df_agg = pd.read_csv("03_output/processed_data/Past_RSV_psv_avg_RLU-uL.csv")

# 1) Subset: protocol = 'ours' and virus = 'Long old'
df_ours_long = df_agg[(df_agg['protocol'] == 'ours') & (df_agg['virus'] == 'Long old')]

# 2) Subset: protocol = 'Cui et al'
df_cui = df_agg[df_agg['protocol'] == 'Cui et al']

# 3) Subset: protocol = 'Hu et al'
df_hu = df_agg[df_agg['protocol'] == 'Hu et al']

# 4) Subset: protocol = 'Haid et al'
df_haid = df_agg[df_agg['protocol'] == 'Haid et al']

## our long on each cell line

In [11]:
import altair as alt

# Color and shape mappings (same as multi-panel plot)
color_mapping = {
    "A549": "#377eb8",
    "Hep2": "#4daf4a",
    "Huh7.5.1": "#e41a1c",
    "293T-TIM1": "#999999"
}
shape_mapping = {
    "A549": "circle",
    "Hep2": "diamond",
    "Huh7.5.1": "triangle",
    "293T-TIM1": "square"
}

# Log ticks and labels
log_ticks = [10**i for i in range(1, 8)]
log_labels = ["10¹", "10²", "10³", "10⁴", "10⁵", "10⁶", "10⁷"]
label_expr = "{ " + ", ".join(f"{v}: '{l}'" for v, l in zip(log_ticks, log_labels)) + " }[datum.value]"

# Base chart with color/shape encodings
base = alt.Chart(df_ours_long).encode(
    x=alt.X(
        'target cell:N',
        title='Target Cell',
        axis=alt.Axis(
            labelAngle=-90,
            labelFontSize=14,
            labelFontWeight='bold',
            titleFontSize=16,
            titleFontWeight='bold'
        )
    ),
    y=alt.Y(
        'mean_RLU_uL:Q',
        title='Pseudovirus Titer (RLU/uL)',
        scale=alt.Scale(type='log', domain=[10, df_ours_long['upper_RLU_uL'].max()]),
        axis=alt.Axis(
            values=log_ticks,
            labelExpr=label_expr,
            labelFontSize=14,
            titleFontSize=16
        )
    ),
    color=alt.Color(
        'target cell:N',
        scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
        legend=alt.Legend(title="Target Cell")
    ),
    shape=alt.Shape(
        'target cell:N',
        scale=alt.Scale(domain=list(shape_mapping.keys()), range=list(shape_mapping.values())),
        legend=None
    ),
    tooltip=[
        alt.Tooltip('target cell:N'),
        alt.Tooltip('mean_RLU_uL:Q'),
        alt.Tooltip('std_RLU_uL:Q'),
    ]
)

# Error bars and caps
error_bars = alt.Chart(df_ours_long).mark_rule(size=2, color='black').encode(
    x='target cell:N',
    y='lower_RLU_uL:Q',
    y2='upper_RLU_uL:Q'
)

error_caps = (
    alt.Chart(df_ours_long).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='target cell:N',
        y='lower_RLU_uL:Q'
    ) +
    alt.Chart(df_ours_long).mark_tick(size=12, thickness=2, orient='horizontal', color='black').encode(
        x='target cell:N',
        y='upper_RLU_uL:Q'
    )
)

# Points (with colored shape per target cell)
points = base.mark_point(size=80, filled=True, stroke='black', strokeWidth=1.5)

# Combine layers
chart = (error_bars + error_caps + points).properties(
    width=200,
    height=150
).configure_axis(
    labelFontWeight='bold',
    titleFontWeight='bold',
    grid=False
).configure_view(
    strokeWidth=2
)

# Save and display
chart.save("03_output/plots/Titer_ours-Long_bycell_colored.html")
chart


alt.LayerChart(...)

## plot past psv protocols

In [16]:
# --- Protocol-specific data and virus filters ---
protocol_data = [
    (df_cui, "Cui et al"),
    (df_hu, "Hu et al"),
    (df_haid, "Haid et al")
]

virus_filters = {
    "Cui et al": ["Bald", "ABJ", "VSVG"],
    "Hu et al": ["Bald", "Long F (full)+G (full)", "VSVG"],
    "Haid et al": ["Bald", "Long F -4+G (full)", "VSVG"]
}
virus_display_names = {
    "Bald": "Bald",
    "ABJ": "A isolate BJ/40180",
    "Long F (full)+G (full)": "Long F full CT+Long G full CT",
    "Long F -4+G (full)": "Long F 4AA CTdel+G full CT",
    "VSVG": "VSV-G"
}

# --- Enforce consistent virus order ---
virus_order = list(virus_display_names.keys())
virus_order_dict = {v: i for i, v in enumerate(virus_order)}

# --- Color and shape mappings ---
color_mapping = {
    "A549": "#377eb8",
    "Hep2": "#4daf4a",
    "Huh7.5.1": "#e41a1c"
}
shape_mapping = {
    "A549": "circle",
    "Hep2": "diamond",
    "Huh7.5.1": "triangle"
}

# --- Shared Y scale ---
shared_y_domain = [10, 10**6]
log_ticks = [10**i for i in range(1, 7)]
log_labels = ["10¹", "10²", "10³", "10⁴", "10⁵", "10⁶"]
label_expr_y = "{ " + ", ".join(f"{v}: '{l}'" for v, l in zip(log_ticks, log_labels)) + " }[datum.value]"
label_expr_x = "{ " + ", ".join(f"{i}: '{virus_display_names[v]}'" for i, v in enumerate(virus_order)) + " }[datum.value]"

# --- Build the charts ---
charts = []
for df, title in protocol_data:
    df = df.copy()
    df['virus'] = df['virus'].str.strip()
    df['virus_order'] = df['virus'].map(virus_order_dict)
    allowed_viruses = virus_filters[title]
    df = df[df['virus'].isin(allowed_viruses)]

    base = alt.Chart(df).encode(
        x=alt.X(
            'virus_order:O',
            title='',
            axis=alt.Axis(
                labelAngle=-90,
                labelFontSize=10,
                titleFontSize=14,
                labelExpr=label_expr_x
            )
        ),
        y=alt.Y(
            'mean_RLU_uL:Q',
            title='Pseudovirus Titer (RLU/uL)',
            scale=alt.Scale(type='log', domain=shared_y_domain),
            axis=alt.Axis(
                values=log_ticks,
                labelExpr=label_expr_y,
                titleFontSize=14
            )
        ),
        color=alt.Color(
            'target cell:N',
            scale=alt.Scale(domain=list(color_mapping.keys()), range=list(color_mapping.values())),
            legend=alt.Legend(title="Target Cell")
        ),
        shape=alt.Shape(
            'target cell:N',
            scale=alt.Scale(domain=list(shape_mapping.keys()), range=list(shape_mapping.values())),
            legend=None
        ),
        tooltip=[
            alt.Tooltip('virus:N', title='Virus'),
            alt.Tooltip('target cell:N', title='Target Cell'),
            alt.Tooltip('mean_RLU_uL:Q', title='Titer (RLU/uL)'),
            alt.Tooltip('std_RLU_uL:Q', title='Std Dev')
        ]
    )

    error_bars = alt.Chart(df).mark_rule(size=2, color='black').encode(
        x='virus_order:O',
        y='lower_RLU_uL:Q',
        y2='upper_RLU_uL:Q'
    )

    error_caps = (
        alt.Chart(df).mark_tick(size=10, thickness=2, orient='horizontal', color='black').encode(
            x='virus_order:O',
            y='lower_RLU_uL:Q'
        ) +
        alt.Chart(df).mark_tick(size=10, thickness=2, orient='horizontal', color='black').encode(
            x='virus_order:O',
            y='upper_RLU_uL:Q'
        )
    )

    points = base.mark_point(size=120, filled=True, stroke='black', strokeWidth=1)

    chart = (error_bars + error_caps + points).properties(
        width=160,
        height=150
    )

    charts.append(chart)

# --- Combine side-by-side ---
combined = alt.hconcat(*charts).resolve_scale(
    y='shared',
    color='shared',
    shape='shared'
).configure_axis(
    labelFontWeight='bold',
    titleFontWeight='bold',
    grid=False
).configure_view(
    strokeWidth=2
)

# --- Save and show ---
combined.save("03_output/plots/Titer_Cui-Hu-Haid_filtered.html")
combined

alt.HConcatChart(...)